# Extracting & Preparing data- New York CityBike2022 

In [1]:
import pandas as pd
import numpy as np
import os
import requests
import json
from datetime import datetime

In [ ]:
import sys
!{sys.executable} -m pip install python-dotenv


# Unzipping & Concactonate all months files of 2022 Citybike

Multistep workflow is needed to prepare the files.
Loops mehtod will be used for such complex workflow and not List comprehension.

unzips each month

finds all Excel/CSV files

concatenates them into one monthly DataFrame

saves each month as a single file

finally merges all months into one big 2022 file

In [14]:
# step 1  create a list of all zipped files

from dotenv import load_dotenv
load_dotenv()
base_path = os.getenv("BASE_PATH")

# Create output folders 
monthly_output = os.getenv("MONTHLY_OUTPUT")
final_output = os.getenv("FINAL_OUTPUT")

# List all zip files (each month)
zip_files = [f for f in os.listdir(base_path) if f.endswith(".zip")]
zip_files


['202201-citibike-tripdata.zip',
 '202202-citibike-tripdata.zip',
 '202203-citibike-tripdata.zip',
 '202204-citibike-tripdata.zip',
 '202205-citibike-tripdata.zip',
 '202206-citibike-tripdata.zip',
 '202207-citibike-tripdata.zip',
 '202208-citibike-tripdata.zip',
 '202209-citibike-tripdata.zip',
 '202210-citibike-tripdata.zip',
 '202211-citibike-tripdata.zip',
 '202212-citibike-tripdata.zip']

In [ ]:
# Step 2 — Unzip each month into its own folder
import zipfile

for zip_file in zip_files:
    month_name = zip_file.split("-")[0]  # e.g. "202201"
    month_folder = os.path.join(base_path, month_name)

    # Create folder for extracted files
    os.makedirs(month_folder, exist_ok=True)

    # Extract ZIP
    with zipfile.ZipFile(os.path.join(base_path, zip_file), 'r') as z:
        z.extractall(month_folder)

print("All ZIP files extracted.")


In [ ]:
# Step 3 — Read all CSV/Excel files for each month

monthly_dfs = []  # store each month's combined DataFrame

for zip_file in zip_files:
    month_name = zip_file.split("-")[0]
    month_folder = os.path.join(base_path, month_name)

    # Find all CSV/Excel files
    files = [f for f in os.listdir(month_folder) if f.endswith((".csv", ".xlsx"))]

    month_df_list = []

    for file in files:
        file_path = os.path.join(month_folder, file)

        if file.endswith(".csv"):
            df = pd.read_csv(file_path)
        else:
            df = pd.read_excel(file_path)

        month_df_list.append(df)

    # Combine all files for that month
    monthly_df = pd.concat(month_df_list, ignore_index=True)

    # Save monthly combined file
    monthly_df.to_csv(os.path.join(monthly_output, f"{month_name}_combined.csv"), index=False)

    # Store for final merge
    monthly_dfs.append(monthly_df)

print("Monthly files processed and saved.")


In [41]:
len(monthly_dfs)

12

#### Important Note:
step 3 should be inside the loop that started in step 2.
carfule with the spacing in writing the code in separate cells.
Alternatively write all steps in one cell code , one loop

len(monthly_dfs) # this can be used to test if it performed the loop on all 12 months, 


In [46]:
monthly_dfs[-3].head()


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,B4644A6CDBDE7AB3,electric_bike,2022-10-01 22:27:40.027,2022-10-01 23:03:09.530,E 171 St & 3 Ave,8171.06,E 171 St & 3 Ave,8171.06,40.837182,-73.902245,40.837119,-73.902185,member
1,06B845E544704C90,electric_bike,2022-10-01 23:03:46.784,2022-10-01 23:49:12.812,E 171 St & 3 Ave,8171.06,E 171 St & 3 Ave,8171.06,40.837152,-73.902255,40.837119,-73.902185,member
2,9E07C2CBC2C107CC,classic_bike,2022-10-23 12:12:55.625,2022-10-23 12:28:28.908,E 10 St & 2 Ave,5746.02,Mott St & Prince St,5561.04,40.729708,-73.986598,40.723180,-73.994800,member
3,9EF49C7F4C88052F,classic_bike,2022-10-07 06:44:54.632,2022-10-07 06:46:16.993,Franklin St & Dupont St,5944.01,Franklin St & Dupont St,5944.01,40.735640,-73.958660,40.735640,-73.958660,member
4,1529759A42D7061F,electric_bike,2022-10-04 16:26:45.089,2022-10-04 16:28:33.408,Mott St & Prince St,5561.04,Mott St & Prince St,5561.04,40.723180,-73.994800,40.723180,-73.994800,member


In [ ]:
# Step 4 — Combine all months into one full-year dataset
# ---------------------------------------------------------

full_2022_df = pd.concat(monthly_dfs, ignore_index=True)

# Build the final file path using your hidden FINAL_OUTPUT variable
final_path = os.path.join(final_output, "citibike_2022_full.csv")

full_2022_df.to_csv(final_path, index=False)

print(f"\nFinal 2022 dataset saved to: {final_path}")


In [52]:
full_2022_df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,BFD29218AB271154,electric_bike,2022-01-21 13:13:43.392,2022-01-21 13:22:31.463,West End Ave & W 107 St,7650.05,Mt Morris Park W & W 120 St,7685.14,40.802117,-73.968181,40.804038,-73.945925,member
1,7C953F2FD7BE1302,classic_bike,2022-01-10 11:30:54.162,2022-01-10 11:41:43.422,4 Ave & 3 St,4028.04,Boerum Pl\t& Pacific St,4488.09,40.673746,-73.985649,40.688489,-73.991160,member
2,95893ABD40CED4B8,electric_bike,2022-01-26 10:52:43.096,2022-01-26 11:06:35.227,1 Ave & E 62 St,6753.08,5 Ave & E 29 St,6248.06,40.761227,-73.960940,40.745168,-73.986831,member
3,F853B50772137378,classic_bike,2022-01-03 08:35:48.247,2022-01-03 09:10:50.475,2 Ave & E 96 St,7338.02,5 Ave & E 29 St,6248.06,40.783964,-73.947167,40.745168,-73.986831,member
4,7590ADF834797B4B,classic_bike,2022-01-22 14:14:23.043,2022-01-22 14:34:57.474,6 Ave & W 34 St,6364.10,5 Ave & E 29 St,6248.06,40.749640,-73.988050,40.745168,-73.986831,member


In [51]:
full_2022_df.tail()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
29838801,1F223EDAFF420AE3,electric_bike,2022-12-01 20:26:45.847,2022-12-01 20:30:46.012,Avenue D & E 3 St,5436.09,Stanton St & Chrystie St,5523.02,40.720701,-73.977939,40.722293,-73.991475,member
29838802,CFA5C560ACB73B8E,classic_bike,2022-12-26 13:46:34.237,2022-12-26 13:52:43.900,43 Ave & 47 St,6209.05,39 Ave & 45 St,6401.03,40.744806,-73.917290,40.749478,-73.918265,member
29838803,11C8C5E0DB947B07,classic_bike,2022-12-01 05:56:14.903,2022-12-01 06:06:10.357,Avenue D & E 3 St,5436.09,Bleecker St & Crosby St,5679.08,40.720828,-73.977932,40.726156,-73.995102,member
29838804,5B9B083C534A5964,classic_bike,2022-12-02 11:54:15.871,2022-12-02 12:01:00.747,Montague St & Clinton St,4677.06,Sands St & Jay St,4821.03,40.694271,-73.992327,40.700119,-73.986200,member
29838805,91C286C462F89A50,classic_bike,2022-12-18 13:35:22.574,2022-12-18 13:37:27.193,Montague St & Clinton St,4677.06,Cadman Plaza E & Tillary St,4677.01,40.694271,-73.992327,40.695977,-73.990149,member
